In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [2]:
data = pd.read_csv("data/GFM_dataRevised.csv", sep = "\t")
print(data.shape)
data.head()

(17111, 11)


,Url,Category,Position,Title,Location,Organiser,Amount_Raised,Goal,Duration,Text,Image
0,https://www.gofundme.com/f/money-to-buy-hearin...,Medical,0,Fundraiser by Chandan Shivu Kumar : Money to b...,NaN,NaN,$250,$250,236.0,"Hello there, I'm a random guy from India who's...",data/images/44793298_1579175168658123_r.jpeg
1,https://www.gofundme.com/f/justiceforjacobblake,Medical,1,Fundraiser by Julia Jackson : Justice for Jaco...,"Kenosha, WI",Julia Jackson,"$2,252,260","$3,000,000",15.0,On August 23rd my son was shot multiple times ...,data/images/50669212_1598282954644748_r.jpg
2,https://www.gofundme.com/f/official-navajo-nat...,Medical,2,Fundraiser for Pearline Kirk by Navajo Nation ...,"Window Rock, AZ",Navajo Nation,"$1,770,760","$1,000,000",127.0,The Navajo Nation COVID-19 Fund has been estab...,data/images/47855360_1589149838795170_r.jpeg
3,https://www.gofundme.com/f/emergenza-covid-ces...,Medical,2,Fundraiser by Gloria Zavatta : Cesvi per l'osp...,Team member,Gloria Zavatta,"€1,506,210","€1,500,000",182.0,Per aggiornamenti su come stiamo impiegando i ...,data/images/46073102_1583841007387069_r.jpeg
4,https://www.gofundme.com/f/save-nil039s-life,Medical,2,Fundraiser by Berk Atikoğlu : Save Nil's Life!,"Brooklyn, NY",Berk Atikoğlu,"$2,402,980","$2,400,000",182.0,Save Nil’s Life ! I am starting a campaign for...,data/images/46072754_1583838455417877_r.jpeg


Number of fundraiser's with amount_raised = goal


(Check if amount raised = 0)

In [14]:
_ = data[data.Amount_Raised == data.Goal]
print(len(_))

266


In [15]:
print(_.Url.iloc[7])
print(_.Url.iloc[0])

https://www.gofundme.com/f/help-kuba-go-to-seamajor-and-first-attack-for-dbfz
https://www.gofundme.com/f/money-to-buy-hearing-aids-for-me-as-well-as-others


In [16]:
headers = ["Url", "Category","Position", "Title", "Location", "Organiser","Amount_Raised", "Goal", "Duration", "Text", "Image"]

In [17]:
listInvalidCampaign = []
lstScrapingError = []

def scrape_url(url, tid = "mainThread"):
    print("Scraping url %s", url)
        
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    
    temp_df = pd.DataFrame(columns = headers)
    
#     invalidCampaign = soup.find('div', attrs={'class': 'm-campaign-sidebar-notification m-well m-well--sand'})
    
#     if(invalidCampaign is not None):
#         text_container =  soup.find('meta', attrs={'name': 'description'})
#         print(text_container['content'])
#         listInvalidCampaign.append(row_index)
#         return "Invalid Campaign"
    
    try:
        # Description
        text_container =  soup.find('meta', attrs={'name': 'description'})
        try:
            all_text = text_container['content']
        except:
            all_text = np.nan

        if(all_text is np.nan )or(all_text == "We're sorry, but that campaign cannot be found. Please check the link URL and try again."):
            print(tid+' ('+str(row_index)+') -> '+all_text)
            listInvalidCampaign.append(row_index)
            temp_df = pd.DataFrame(columns = headers)
            return(temp_df)

        # Title
        try:
            title = soup.find("title").text
        except:
            title = np.nan

        # Organiser & Location
        try:
            t = soup.find_all("div",{"class":"p-campaign-members"})[0]
            organiser = t.find_all("div",{"class": "m-person-info-name"})[0].text.replace('\xa0','')
            location = ""
            if(len(t.find_all("div",{"class": "text-small"})) > 3):
                location = t.find_all("div",{"class": "text-small"})[2].text.replace('\xa0','')
            else:
                location = t.find_all("div",{"class": "text-small"})[1].text.replace('\xa0','')
        except:
            organiser = np.nan
            location = np.nan

        # Amount Raised and Goal
        try:
            t = soup.find("div",{"class":"p-campaign-sidebar"})
            amount = t.find("div",{"class": "o-campaign-sidebar-progress-meter m-progress-meter"})
            amountRaised = amount.text.split()[0]
            goal = amount.text.split()[-2]
        except:
            amountRaised = np.nan
            goal = np.nan

        # All Text
        soup.find('div', {'class': "p-campaign-description"})
        text = soup.find('div', {'class': "o-campaign-story mt-three-halves"}).text.replace('\xa0','').replace('\n','')
        text

        # Duration of Fundraising
        try:
            t = soup.find("div",{"class": "p-campaign-description"})
            startDate = ' '.join(t.find("li",{"class": "m-meta-list-item"}).text.split()[1::])
            startDate = (pd.to_datetime(startDate)).date()
            duration = (pd.datetime.now().date() - startDate).days
        except:
            duration = np.nan

        #Image
        try:
            img_container =  soup.find('meta', attrs={'property': 'og:image'})
            image_url = img_container['content']

            img_filename = "data/images/"+image_url.split("/")[-1]

            # Open the url image, set stream to True, this will return the stream content.
            r = requests.get(image_url, stream = True)

            # Check if the image was retrieved successfully
            if r.status_code == 200:
                # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
                r.raw.decode_content = True

                # Open a local file with wb ( write binary ) permission.
                with open(img_filename,'wb') as f:
                    shutil.copyfileobj(r.raw, f)
        except:
            img_filename = np.nan

    #     # FB likes and Shares    
    #     try:
    #         FB_shares_container = soup.find_all("strong", {"class":"js-share-count-text"})
    #         FB_shares = FB_shares_container[0].text.splitlines()
    #         FB_shares = FB_shares[1].replace(" ", "").replace("\xa0", "")
    #     except:
    #         FB_shares = np.nan

    #     try:
    #         hearts_container = soup.find_all("div", {"class":"campaign-sp campaign-sp--heart fave-num"})
    #         hearts = hearts_container[0].text
    #     except:
    #         hearts = np.nan

        temp_row = np.array([[url, category, position, title, location, organiser, amountRaised, goal, duration, text, img_filename]])
        temp_df = pd.DataFrame(temp_row, columns = headers)

        return(temp_df)
    
    except:
        lstScrapingError.append(row_index)
        return tid+' ('+str(row_index)+') -> '+"Scraping Error"

In [18]:
scrape_url(_.Url.iloc[7])

Scraping url %s https://www.gofundme.com/f/help-kuba-go-to-seamajor-and-first-attack-for-dbfz


NameError: name 'requests' is not defined